In [1]:
# Keras' imports
from keras import models, layers, optimizers
from keras.layers import AveragePooling2D, MaxPooling2D, Lambda, MaxPool2D
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, BatchNormalization
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Model, Sequential, model_from_json
print("Keras imported")

# Pre-trained models
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet import ResNet50
print("Models imported")

# Callbacks
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Sklearn's imports
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve, KFold, cross_val_score, StratifiedKFold
from sklearn.utils import class_weight, shuffle
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc
print("Sklearn imported")

import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
print("All libraries imported sucessfully!!")

Using TensorFlow backend.
/home/ayush-bansal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ayush-bansal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ayush-bansal/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ayush-bansal/.local/lib/python3.6/site-pa

Keras imported


ModuleNotFoundError: No module named 'keras.applications.resnet'

In [ ]:
from google.colab import drive
path = '/content/drive'
drive.mount(path, force_remount=True)

root = path + '/My Drive/'
train_dir = root + "Train/"
test_dir =  root + "Test/"

Vgg16_notop        = root + 'vgg16_notop.h5'
Xception_notop     = root + 'xception_notop.h5'
Inception_v2_notop = root + 'Weights/inception_resnet_v2_notop.h5'
Inception_v3_notop = root + 'inception_v3_notop.h5'
Resnet50_notop     = root + 'resnet50_notop.h5'

Xception_top       = root + 'xception_top.h5'
Inception_v2_top   = root + 'inception_resnet_v2_top.h5'
Inception_v3_top   = root + 'inception_v3_top.h5'
Resnet50_top       = root + 'resnet50_top.h5'

In [ ]:
dict_characters = {}
imageSize = 60

def get_data(folder):
    # Load the data and classifing labels from the given folder.
    X = []
    y = []
    for folderName in os.listdir(folder):
        folderName = folderName.split("_")[0]
        
        # finding label for a image
        if folderName == 'del':
            label = 27
        elif folderName == 'nothing':
            label = 29
        elif folderName == 'spacebar':
            label = 28           
        elif len(folderName) == 1:
            label = ord(folderName) - 64
        
        # reading name of images 
        if folder.endswith("test/"):
            image_filenames = [folder + folderName + "_test.jpg"]
        else:
            map_characters[label] = folderName
            image_filenames = os.listdir(folder + folderName)
        
        # Making image vector
        for image_filename in image_filenames:
            img_file = cv2.imread(folder + folderName + '/' + image_filename)
            img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
            img_arr = np.asarray(img_file)
            X.append(img_arr)
            y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X, y
X_train, y_train = get_data(train_dir)
#X_test, y_test= get_data(test_dir) # Too few images

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.25) 

# Encode labels to hot vectors
y_trainHot = to_categorical(y_train, num_classes = 30)
y_testHot = to_categorical(y_test, num_classes = 30)

In [ ]:
# Shuffle data to permit further subsampling
# Max Value can be 87000 = 29 * 3000[(no. of classes) * (no of images)]
sample_size = 2397
X_train, y_trainHot = shuffle(X_train, y_trainHot, random_state = 2397)
X_test, y_testHot = shuffle(X_test, y_testHot, random_state = 2397)

X_train = X_train[ : sample_size]
X_test = X_test[ : sample_size]
y_trainHot = y_trainHot[ : sample_size]
y_testHot = y_testHot[ : sample_size]

In [ ]:
# Helper Functions Learning Curves and Confusion Matrix
class MetricsCheckpoint(Callback):
    """Callback that saves metrics after each epoch"""
    def __init__(self, savepath):
        super(MetricsCheckpoint, self).__init__()
        self.savepath = savepath
        self.history = {}
    def on_epoch_end(self, epoch, logs=None):
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        np.save(self.savepath, self.history)

def plotKerasLearningCurve():
    plt.figure(figsize=(10,5))
    metrics = np.load('logs.npy')[()]
    filt = ['acc'] 
    # try to add 'loss' to see the loss learning curve
    for k in filter(lambda x : np.any([kk in x for kk in filt]), metrics.keys()):
        l = np.array(metrics[k])
        plt.plot(l, c= 'r' if 'val' not in k else 'b', label='val' if 'val' in k else 'train')
        x = np.argmin(l) if 'loss' in k else np.argmax(l)
        y = l[x]
        plt.scatter(x,y, lw=0, alpha=0.25, s=100, c='r' if 'val' not in k else 'b')
        plt.text(x, y, '{} = {:.4f}'.format(x,y), size='15', color= 'r' if 'val' not in k else 'b')   
    plt.legend(loc=4)
    plt.axis([0, None, None, None]);
    plt.grid()
    plt.xlabel('Number of epochs')
    plt.ylabel('Accuracy')

def plot_confusion_matrix(cm, classes, normalize = False, title = 'Confusion matrix', cmap = plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize = (8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot_learning_curve(history):
    plt.figure(figsize=(8,8))
    plt.subplot(1,2,1)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(root + 'plots/accuracy_curve.png')
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig(root + 'plots/loss_curve.png')

In [ ]:
def topless_Network(data, pretrainedmodel, pretrainedweights, classweight, optimizer, labels):
    xtrain, ytrain, xtest, ytest = data
    base_model = pretrainedmodel
    
    # Add top layer
    x = base_model.output
    x = Flatten()(x)
    predictions = Dense(30, activation='softmax')(x)
    model = Model(inputs = base_model.input, outputs = predictions)
    
    # Train top layer
    for layer in base_model.layers:
        layer.trainable = False
    
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])
    callbacks_list = [EarlyStopping(monitor = 'val_acc', patience = 3, verbose = 1)]
    model.summary()
    
    # Fit model
    history = model.fit(xtrain, ytrain, 
                        epochs = 10, 
                        class_weight = classweight, 
                        validation_data = (xtest, ytest), 
                        verbose = 1, 
                        callbacks = [MetricsCheckpoint(root + 'logs')])
    
    # Evaluate model
    score = model.evaluate(xtest, ytest, verbose = 0)
    print('\nKeras CNN - accuracy:', score[1], '\n')
    y_pred = model.predict(xtest)
    class_report = classification_report(np.where(ytest > 0)[1], 
                                         np.argmax(y_pred, axis=1), 
                                         target_names = list(labels.values()))
    print('\n', class_report. sep = "") 
    Y_pred_classes = np.argmax(y_pred,axis = 1) 
    Y_true = np.argmax(ytest, axis = 1) 
    confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
    
    # Ploting results
    plotKerasLearningCurve()
    plt.show()
    plot_learning_curve(history)
    plt.show()
    plot_confusion_matrix(confusion_mtx, classes = list(labels.values()))
    plt.show()
    return model

# Weights
class_weight1 = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

# Pre-Trained model
model_1 = VGG16(weights = Vgg16_notop, 
                include_top = False, 
                input_shape = (imageSize, imageSize, 3))

model_2 = InceptionV3(weights = Inception_v3_notop, 
                      include_top = False, 
                      input_shape = (imageSize, imageSize, 3))

model_3 = Xception(weights = Xception_notop, 
                   include_top = False, 
                   input_shape = (imageSize, imageSize, 3))

model_4 = InceptionResNetV2(weights = Inception_v2_notop, 
                            include_top = False, 
                            input_shape = (imageSize, imageSize, 3))

model_5 = ResNet50(weights = Resnet50_notop, 
                   include_top = False, 
                   input_shape = (imageSize, imageSize, 3))

# Optimizer
optimizer1 = Adam()
optimizer2 = RMSprop(lr = 0.0001)

# Data
data = (X_train, y_trainHot, X_test, y_testHot)

topless_Network(data, model_1, weight_path1, class_weight1, optimizer1, map_characters)

In [ ]:
def model_Network(data, pretrainedmodel, pretrainedweights, classweight, optimizer, labels):
    xtrain, ytrain, xtest, ytest = data
    base_model = pretrainedmodel
    predictions = Dense(30, activation='softmax')(x)
    model = Model(inputs = base_model.input, outputs = predictions)
    
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])
    callbacks_list = [EarlyStopping(monitor = 'val_acc', patience = 3, verbose = 1)]
    model.summary()
    
    # Fit model
    history = model.fit(xtrain, ytrain, 
                        epochs = 10, 
                        class_weight = classweight, 
                        validation_data = (xtest, ytest), 
                        verbose = 1, 
                        callbacks = [MetricsCheckpoint(root + 'logs')])
    
    # Evaluate model
    score = model.evaluate(xtest, ytest, verbose = 0)
    print('\nKeras CNN - accuracy:', score[1], '\n')
    y_pred = model.predict(xtest)
    class_report = classification_report(np.where(ytest > 0)[1], 
                                         np.argmax(y_pred, axis=1), 
                                         target_names = list(labels.values()))
    print('\n', class_report. sep = "") 
    Y_pred_classes = np.argmax(y_pred,axis = 1) 
    Y_true = np.argmax(ytest, axis = 1) 
    confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
    
    # Ploting results
    plotKerasLearningCurve()
    plt.show()
    plot_learning_curve(history)
    plt.show()
    plot_confusion_matrix(confusion_mtx, classes = list(labels.values()))
    plt.show()
    return model

# Weights
class_weight1 = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

# Pre-Trained model
model_1 = VGG16(weights = Vgg16_notop, 
                include_top = True, 
                input_shape = (imageSize, imageSize, 3),
                classes = 30)

model_2 = InceptionV3(weights = Inception_v3_notop, 
                      include_top = True, 
                      input_shape = (imageSize, imageSize, 3),
                      classes = 30)

model_3 = Xception(weights = Xception_notop, 
                   include_top = True, 
                   input_shape = (imageSize, imageSize, 3),
                   classes = 30)

model_4 = InceptionResNetV2(weights = Inception_v2_notop, 
                            include_top = True, 
                            input_shape = (imageSize, imageSize, 3),
                            classes = 30)

model_5 = ResNet50(weights = Resnet50_notop, 
                   include_top = True, 
                   input_shape = (imageSize, imageSize, 3), 
                   classes = 30)

# Optimizer
optimizer1 = Adam()
optimizer2 = RMSprop(lr = 0.0001)

# Data
data = (X_train, y_trainHot, X_test, y_testHot)

model_Network(data, model_1, weight_path1, class_weight1, optimizer1, map_characters)